# Hierarchical Map Generation with 64× Zoom
## Two-Stage Generation: Reasoning → Image

**Workflow:**
1. **Stage 1 (Reasoning)**: Gemini 2.5 Flash Thinking (temp=0) auto-generates prompt from coordinates + world lore
2. **Stage 2 (Image)**: Gemini 2.5 Flash Image generates the map tile

**Features:**
- World lore separate from visual style instructions
- 64× zoom levels only (powers of 2)
- First generation is always the whole world
- Overlap validation between zoom levels
- Auto-crop parent maps with gray borders (bottom+right only)

## Setup

In [27]:
!pip install requests pillow numpy matplotlib python-dotenv scikit-image

  Using cached imageio-2.37.0-py3-none-any.whl.metadata (5.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 12.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [scikit-image] [scikit-image]


In [28]:
import requests
import json
import base64
from PIL import Image, ImageDraw
import io
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import os
from dotenv import load_dotenv
from skimage.metrics import structural_similarity as ssim

load_dotenv()
os.makedirs('outputs', exist_ok=True)

print("✓ Imports loaded")

✓ Imports loaded


In [ ]:
# Configuration
OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY', 'YOUR_API_KEY_HERE')
API_URL = "https://openrouter.ai/api/v1/chat/completions"

# Models
REASONING_MODEL = "google/gemini-2.5-flash"  # For prompt generation
IMAGE_MODEL = "google/gemini-2.5-flash-image"         # For image generation

# Generation params
REASONING_TEMPERATURE = 0  # Deterministic prompt generation
IMAGE_ASPECT_RATIO = "1:1"  # Always square tiles
OUTPUT_RESOLUTION = 1024    # Always 1024×1024 pixels

# Zoom configuration
ZOOM_FACTOR = 64  # 64× zoom between levels

# Border config
BORDER_COLOR = (128, 128, 128)  # Gray color for overflow border
BORDER_WIDTH_PX = 100  # Border width in pixels (bottom and right only)

print(f"Reasoning Model: {REASONING_MODEL}")
print(f"Image Model: {IMAGE_MODEL}")
print(f"Reasoning Temperature: {REASONING_TEMPERATURE}")
print(f"Zoom Factor: {ZOOM_FACTOR}×")
print(f"Border: {BORDER_WIDTH_PX}px on bottom+right only")
print(f"API Key: {'✓ Configured' if OPENROUTER_API_KEY != 'YOUR_API_KEY_HERE' else '✗ Not set'}")

Reasoning Model: google/gemini-2.5-flash-thinking
Image Model: google/gemini-2.5-flash-image
Reasoning Temperature: 0
Zoom Factor: 64×
Border: 100px on bottom+right only
API Key: ✓ Configured


## World Bible (Lore Only)

Contains ONLY world lore - geography, settlements, culture.  
**No visual style specifications** - those are in DEFAULT_VISUAL_INSTRUCTIONS.

In [30]:
WORLD_BIBLE = """
# World: Eldermyst

## Geography
- Temperate continent with rolling hills and mountain ranges
- Major river (Silverflow) runs through central valley
- Fertile plains in the south, dense forests in the north
- Coastal cities along the eastern shore
- Mountain peaks with eternal snow in the western range

## Settlements
- **Capital**: Thornhaven - walled city on Silverflow River
- **Villages**: Timber and stone construction, red clay roofs
- **Roads**: Cobblestone in cities, dirt paths in countryside
- **Bridges**: Stone arch bridges over waterways

## Architecture
- Medieval fantasy aesthetic
- Timber-framed buildings with stone foundations
- Steep gabled roofs with red/brown clay tiles
- Small windows with shutters
- Town squares with central fountains or wells

## Natural Features
- **Trees**: Oak, pine, birch in forests
- **Grass**: Vibrant green in plains, darker in forests
- **Water**: Clear blue rivers and streams
- **Mountains**: Rocky gray stone with snow-capped peaks
"""

print("World Bible Loaded")
print("=" * 70)
print(WORLD_BIBLE.strip())
print("=" * 70)

World Bible Loaded
# World: Eldermyst

## Geography
- Temperate continent with rolling hills and mountain ranges
- Major river (Silverflow) runs through central valley
- Fertile plains in the south, dense forests in the north
- Coastal cities along the eastern shore
- Mountain peaks with eternal snow in the western range

## Settlements
- **Capital**: Thornhaven - walled city on Silverflow River
- **Villages**: Timber and stone construction, red clay roofs
- **Roads**: Cobblestone in cities, dirt paths in countryside
- **Bridges**: Stone arch bridges over waterways

## Architecture
- Medieval fantasy aesthetic
- Timber-framed buildings with stone foundations
- Steep gabled roofs with red/brown clay tiles
- Small windows with shutters
- Town squares with central fountains or wells

## Natural Features
- **Trees**: Oak, pine, birch in forests
- **Grass**: Vibrant green in plains, darker in forests
- **Water**: Clear blue rivers and streams
- **Mountains**: Rocky gray stone with snow-capp

## Default Visual Instructions

Visual style specifications that get appended to ALL generated prompts.

In [31]:
DEFAULT_VISUAL_INSTRUCTIONS = """
## Visual Style Requirements
- 16-bit pixel art (SNES era aesthetic - Chrono Trigger, Final Fantasy VI)
- Top-down orthogonal view (no perspective)
- Clean pixels with dithering for texture
- Consistent lighting from top-left
- Seamless edges for tiling
- Gray border on BOTTOM and RIGHT edges only
- Objects can extend into gray border area
- Skip/cut objects that would extend past TOP or LEFT edges
"""

print("Default Visual Instructions:")
print(DEFAULT_VISUAL_INSTRUCTIONS.strip())

Default Visual Instructions:
## Visual Style Requirements
- 16-bit pixel art (SNES era aesthetic - Chrono Trigger, Final Fantasy VI)
- Top-down orthogonal view (no perspective)
- Clean pixels with dithering for texture
- Consistent lighting from top-left
- Seamless edges for tiling
- Gray border on BOTTOM and RIGHT edges only
- Objects can extend into gray border area
- Skip/cut objects that would extend past TOP or LEFT edges


## Utility Functions

In [32]:
def encode_image(image):
    """Encode PIL Image to base64 data URL"""
    if isinstance(image, str):
        with open(image, 'rb') as f:
            image_data = f.read()
    else:
        buffer = io.BytesIO()
        image.save(buffer, format='PNG')
        image_data = buffer.getvalue()
    
    base64_image = base64.b64encode(image_data).decode('utf-8')
    return f"data:image/png;base64,{base64_image}"

def decode_image(base64_string):
    """Decode base64 data URL to PIL Image"""
    if base64_string.startswith('data:image'):
        base64_string = base64_string.split(',')[1]
    image_data = base64.b64decode(base64_string)
    return Image.open(io.BytesIO(image_data))

def save_image(base64_or_image, filepath):
    """Save image to file"""
    if isinstance(base64_or_image, str):
        img = decode_image(base64_or_image)
    else:
        img = base64_or_image
    
    img.save(filepath)
    print(f"💾 Saved: {filepath}")
    return img

def display_images(images, titles=None, figsize=(18, 6)):
    """Display multiple images"""
    n = len(images)
    fig, axes = plt.subplots(1, n, figsize=figsize)
    if n == 1:
        axes = [axes]
    
    for i, (ax, img) in enumerate(zip(axes, images)):
        if isinstance(img, str):
            img = decode_image(img)
        ax.imshow(img)
        ax.axis('off')
        if titles and i < len(titles):
            ax.set_title(titles[i], fontsize=10)
    
    plt.tight_layout()
    plt.show()

print("✓ Utility functions loaded")

✓ Utility functions loaded


## Core Generation Functions

In [33]:
PROMPT_GENERATION_TEMPLATE = """
You are a map generation prompt engineer. Generate a detailed image generation prompt.

**World Context:**
{bible}

**Generation Type:** {generation_type}

**Section Coordinates:**
- Center Position: ({x_m}m, {y_m}m) from world origin
- Size: {width_m} meters × {height_m} meters
- Output Resolution: {output_px}×{output_px} pixels (square)
- Scale: 1 pixel = {meters_per_pixel:.4f} meters
{parent_context}

**Your Task:**
Based on the world lore and the coordinates/scale provided:
1. Determine what terrain, settlements, or features would logically appear at this location
2. Describe appropriate level of detail for this scale ({meters_per_pixel:.4f} m/px)
3. Generate a detailed prompt for image generation

{is_world_instruction}

Output ONLY the image generation prompt, no explanation or commentary.
"""

def generate_section_prompt(bible, coordinate, is_world=False, parent_map=None):
    """Stage 1: Generate detailed prompt using reasoning model
    
    Args:
        bible: World lore (no visual style)
        coordinate: {x_m, y_m, width_m, height_m} in meters
        is_world: True if generating whole world, False if zooming into section
        parent_map: Optional parent map image (for context when is_world=False)
    
    Returns:
        Generated prompt string
    """
    
    # Calculate scale
    meters_per_pixel = coordinate['width_m'] / OUTPUT_RESOLUTION
    
    generation_type = "WHOLE WORLD" if is_world else "ZOOMED SECTION"
    
    parent_context = ""
    if parent_map and not is_world:
        parent_context = "\n- Parent Map: A lower-resolution map of the larger area is provided as reference\n- Use parent map to maintain consistency in terrain and features"
    
    is_world_instruction = ""
    if is_world:
        is_world_instruction = "This is the ENTIRE WORLD - show the full continent with all major geographic features."
    else:
        is_world_instruction = "This is a ZOOMED SECTION - show detailed features at this specific location."
    
    prompt = PROMPT_GENERATION_TEMPLATE.format(
        bible=bible,
        generation_type=generation_type,
        x_m=coordinate['x_m'],
        y_m=coordinate['y_m'],
        width_m=coordinate['width_m'],
        height_m=coordinate['height_m'],
        output_px=OUTPUT_RESOLUTION,
        meters_per_pixel=meters_per_pixel,
        parent_context=parent_context,
        is_world_instruction=is_world_instruction
    )
    
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "model": REASONING_MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": REASONING_TEMPERATURE
    }
    
    print(f"🧠 Generating prompt for {coordinate['width_m']}m × {coordinate['height_m']}m section...")
    print(f"   Type: {generation_type}")
    print(f"   Scale: 1px = {meters_per_pixel:.4f}m")
    
    response = requests.post(API_URL, headers=headers, json=payload)
    
    if response.status_code != 200:
        print(f"❌ Error {response.status_code}: {response.text}")
        return None
    
    result = response.json()
    
    if result.get("choices") and result["choices"][0]["message"].get("content"):
        generated_prompt = result["choices"][0]["message"]["content"]
        print(f"   ✓ Prompt generated ({len(generated_prompt)} chars)")
        return generated_prompt
    else:
        print("   ❌ No response from reasoning model")
        return None

print("✓ Prompt generation function loaded")

✓ Prompt generation function loaded


In [34]:
def crop_parent_with_borders(parent_map, child_coordinate, parent_coordinate):
    """Crop parent map to section and add gray borders on bottom+right
    
    Args:
        parent_map: PIL Image of parent map
        child_coordinate: {x_m, y_m, width_m, height_m} for child section
        parent_coordinate: {x_m, y_m, width_m, height_m} for parent map
    
    Returns:
        Cropped image with gray borders on bottom and right
    """
    
    # Calculate parent scale
    parent_scale_m_per_px = parent_coordinate['width_m'] / OUTPUT_RESOLUTION
    
    # Convert child coordinates (relative to world origin) to parent map pixels
    # Child position relative to parent's top-left corner
    child_rel_x = child_coordinate['x_m'] - parent_coordinate['x_m']
    child_rel_y = child_coordinate['y_m'] - parent_coordinate['y_m']
    
    # Convert to pixels in parent map
    x_px = int(child_rel_x / parent_scale_m_per_px)
    y_px = int(child_rel_y / parent_scale_m_per_px)
    width_px = int(child_coordinate['width_m'] / parent_scale_m_per_px)
    height_px = int(child_coordinate['height_m'] / parent_scale_m_per_px)
    
    # Crop the section
    cropped = parent_map.crop((x_px, y_px, x_px + width_px, y_px + height_px))
    
    # Add gray borders on bottom and right
    bordered_width = cropped.width + BORDER_WIDTH_PX
    bordered_height = cropped.height + BORDER_WIDTH_PX
    
    bordered = Image.new('RGB', (bordered_width, bordered_height), BORDER_COLOR)
    bordered.paste(cropped, (0, 0))
    
    print(f"   📐 Cropped from parent: ({x_px},{y_px}) size ({width_px}×{height_px})")
    print(f"   🎨 Added {BORDER_WIDTH_PX}px gray border (bottom+right only)")
    
    return bordered

print("✓ Cropping function loaded")

✓ Cropping function loaded


In [35]:
def generate_section_image(generated_prompt, visual_instructions, bible, parent_map_image=None):
    """Stage 2: Generate image using generated prompt + visual instructions
    
    Args:
        generated_prompt: Prompt from Stage 1 (what to generate)
        visual_instructions: How to render it (16-bit, top-down, etc)
        bible: World lore
        parent_map_image: Optional cropped parent map with borders
    
    Returns:
        base64 data URL of generated image
    """
    
    # Combine: bible + generated prompt + visual instructions
    full_prompt = f"{bible}\n\n---\n\n{generated_prompt}\n\n---\n\n{visual_instructions}"
    
    # Build message content
    message_content = [{"type": "text", "text": full_prompt}]
    
    # Add parent map if provided
    if parent_map_image:
        message_content.append({
            "type": "image_url",
            "image_url": {"url": encode_image(parent_map_image)}
        })
    
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "model": IMAGE_MODEL,
        "messages": [{"role": "user", "content": message_content}],
        "modalities": ["image", "text"],
        "image_config": {"aspect_ratio": IMAGE_ASPECT_RATIO}
    }
    
    print(f"🎨 Generating image...")
    if parent_map_image:
        print(f"   📍 Using parent map as context")
    
    response = requests.post(API_URL, headers=headers, json=payload)
    
    if response.status_code != 200:
        print(f"❌ Error {response.status_code}: {response.text}")
        return None
    
    result = response.json()
    
    if result.get("choices") and result["choices"][0]["message"].get("images"):
        image_url = result["choices"][0]["message"]["images"][0]["image_url"]["url"]
        print("   ✓ Image generated")
        return image_url
    else:
        print("   ❌ No image in response")
        if result.get("choices") and result["choices"][0]["message"].get("content"):
            print(f"   Text: {result['choices'][0]['message']['content']}")
        return None

print("✓ Image generation function loaded")

✓ Image generation function loaded


In [36]:
def generate_map_section(bible, visual_instructions, coordinate, is_world=False, 
                         parent_map=None, parent_coordinate=None):
    """Complete two-stage generation pipeline
    
    Args:
        bible: World lore (no visual style)
        visual_instructions: Visual style specs (16-bit, top-down, etc)
        coordinate: {x_m, y_m, width_m, height_m} in meters
        is_world: True if generating whole world, False if section
        parent_map: PIL Image of parent (required if is_world=False)
        parent_coordinate: {x_m, y_m, width_m, height_m} of parent (required if is_world=False)
    
    Returns:
        (generated_image, generated_prompt, coordinate) tuple
    """
    
    print("\n" + "="*70)
    if is_world:
        print(f"GENERATING WORLD: {coordinate['width_m']}m × {coordinate['height_m']}m")
    else:
        print(f"GENERATING SECTION: {coordinate['width_m']}m × {coordinate['height_m']}m")
        print(f"Position: ({coordinate['x_m']}m, {coordinate['y_m']}m)")
    print("="*70)
    
    # Stage 1: Generate prompt
    prompt = generate_section_prompt(
        bible=bible,
        coordinate=coordinate,
        is_world=is_world,
        parent_map=parent_map
    )
    
    if not prompt:
        return None, None, coordinate
    
    print(f"\n📝 Generated Prompt:")
    print("-" * 70)
    print(prompt)
    print("-" * 70)
    
    # Crop parent if provided
    parent_cropped = None
    if parent_map and parent_coordinate and not is_world:
        parent_cropped = crop_parent_with_borders(parent_map, coordinate, parent_coordinate)
    
    # Stage 2: Generate image
    image = generate_section_image(
        generated_prompt=prompt,
        visual_instructions=visual_instructions,
        bible=bible,
        parent_map_image=parent_cropped
    )
    
    return image, prompt, coordinate

print("✓ Complete pipeline function loaded")

✓ Complete pipeline function loaded


## Overlap Validation

Check consistency between child image and downsampled parent crop.

In [37]:
def validate_zoom_consistency(child_image, parent_map, child_coordinate, parent_coordinate):
    """Validate that zoomed image is consistent with parent
    
    Downsamples child to parent resolution and compares with parent crop.
    
    Args:
        child_image: High-resolution zoomed image
        parent_map: Parent map image
        child_coordinate: {x_m, y_m, width_m, height_m} of child
        parent_coordinate: {x_m, y_m, width_m, height_m} of parent
    
    Returns:
        (ssim_score, parent_crop, downsampled_child, diff_image)
    """
    
    # Crop parent to same region as child
    parent_scale_m_per_px = parent_coordinate['width_m'] / OUTPUT_RESOLUTION
    
    child_rel_x = child_coordinate['x_m'] - parent_coordinate['x_m']
    child_rel_y = child_coordinate['y_m'] - parent_coordinate['y_m']
    
    x_px = int(child_rel_x / parent_scale_m_per_px)
    y_px = int(child_rel_y / parent_scale_m_per_px)
    width_px = int(child_coordinate['width_m'] / parent_scale_m_per_px)
    height_px = int(child_coordinate['height_m'] / parent_scale_m_per_px)
    
    parent_crop = parent_map.crop((x_px, y_px, x_px + width_px, y_px + height_px))
    
    # Downsample child to parent resolution
    if isinstance(child_image, str):
        child_image = decode_image(child_image)
    
    downsampled_child = child_image.resize((width_px, height_px), Image.LANCZOS)
    
    # Convert to numpy arrays for comparison
    parent_array = np.array(parent_crop)
    child_array = np.array(downsampled_child)
    
    # Calculate SSIM (structural similarity)
    ssim_score = ssim(parent_array, child_array, channel_axis=2, data_range=255)
    
    # Create difference image
    diff = np.abs(parent_array.astype(float) - child_array.astype(float))
    diff = (diff / diff.max() * 255).astype(np.uint8)
    diff_image = Image.fromarray(diff)
    
    print(f"\n📊 Overlap Validation:")
    print(f"   SSIM Score: {ssim_score:.4f} (1.0 = perfect match)")
    print(f"   Parent crop size: {width_px}×{height_px}px")
    
    return ssim_score, parent_crop, downsampled_child, diff_image

print("✓ Validation function loaded")

✓ Validation function loaded


## Test 1: Generate Whole World

First generation is ALWAYS the entire world (e.g., 65,536m × 65,536m = 64km × 64km)

In [38]:
# World coordinate - center at (0, 0), 64km × 64km
WORLD_SIZE_M = 65536  # 64km × 64km

world_coordinate = {
    'x_m': 0,
    'y_m': 0,
    'width_m': WORLD_SIZE_M,
    'height_m': WORLD_SIZE_M
}

world_image, world_prompt, _ = generate_map_section(
    bible=WORLD_BIBLE,
    visual_instructions=DEFAULT_VISUAL_INSTRUCTIONS,
    coordinate=world_coordinate,
    is_world=True
)

if world_image:
    world_img = save_image(world_image, "outputs/world_64km.png")
    display_images([world_img], [f"World Map: {WORLD_SIZE_M/1000:.0f}km × {WORLD_SIZE_M/1000:.0f}km\n({WORLD_SIZE_M/OUTPUT_RESOLUTION:.1f} m/px)"])
else:
    print("❌ Failed to generate world")


GENERATING WORLD: 65536m × 65536m
🧠 Generating prompt for 65536m × 65536m section...
   Type: WHOLE WORLD
   Scale: 1px = 64.0000m
❌ Error 400: {"error":{"message":"google/gemini-2.5-flash-thinking is not a valid model ID","code":400},"user_id":"user_2wtt8ihXXTmoL75xGIMqxXKpOhE"}
❌ Failed to generate world


## Test 2: First Zoom (64× Detail)

Zoom into a 1km × 1km section (64× zoom from 64km world)

In [ ]:
if world_image:
    world_img = decode_image(world_image)
    
    # Zoom into center region: 1km × 1km section
    zoom1_size = WORLD_SIZE_M / ZOOM_FACTOR  # 64km / 64 = 1024m = 1km
    
    zoom1_coordinate = {
        'x_m': 0,          # Center of world
        'y_m': 0,          # Center of world
        'width_m': zoom1_size,
        'height_m': zoom1_size
    }
    
    zoom1_image, zoom1_prompt, _ = generate_map_section(
        bible=WORLD_BIBLE,
        visual_instructions=DEFAULT_VISUAL_INSTRUCTIONS,
        coordinate=zoom1_coordinate,
        is_world=False,
        parent_map=world_img,
        parent_coordinate=world_coordinate
    )
    
    if zoom1_image:
        zoom1_img = save_image(zoom1_image, "outputs/zoom1_1km.png")
        
        display_images(
            [world_img, zoom1_img],
            [
                f"World: {WORLD_SIZE_M/1000:.0f}km",
                f"Zoom 1: {zoom1_size/1000:.1f}km (64× detail)"
            ],
            figsize=(14, 7)
        )
        
        # Validate overlap
        ssim_score, parent_crop, downsampled, diff = validate_zoom_consistency(
            zoom1_img, world_img, zoom1_coordinate, world_coordinate
        )
        
        display_images(
            [parent_crop, downsampled, diff],
            ["Parent Crop", "Downsampled Child", f"Difference (SSIM: {ssim_score:.3f})"],
            figsize=(15, 5)
        )
    else:
        print("❌ Failed to generate zoom 1")
else:
    print("⚠️  Skipping Test 2 - no world map")

## Test 3: Second Zoom (4096× Detail from World)

Zoom into a 16m × 16m section (64× zoom from 1km, 4096× from world)

In [ ]:
if 'zoom1_image' in locals() and zoom1_image:
    zoom1_img = decode_image(zoom1_image)
    
    # Zoom into 16m × 16m section
    zoom2_size = zoom1_size / ZOOM_FACTOR  # 1024m / 64 = 16m
    
    zoom2_coordinate = {
        'x_m': 0,          # Center
        'y_m': 0,          # Center
        'width_m': zoom2_size,
        'height_m': zoom2_size
    }
    
    zoom2_image, zoom2_prompt, _ = generate_map_section(
        bible=WORLD_BIBLE,
        visual_instructions=DEFAULT_VISUAL_INSTRUCTIONS,
        coordinate=zoom2_coordinate,
        is_world=False,
        parent_map=zoom1_img,
        parent_coordinate=zoom1_coordinate
    )
    
    if zoom2_image:
        zoom2_img = save_image(zoom2_image, "outputs/zoom2_16m.png")
        
        display_images(
            [world_img, zoom1_img, zoom2_img],
            [
                f"World: {WORLD_SIZE_M/1000:.0f}km",
                f"Zoom 1: {zoom1_size/1000:.1f}km",
                f"Zoom 2: {zoom2_size:.1f}m (4096× from world)"
            ],
            figsize=(18, 6)
        )
        
        # Validate overlap with zoom1
        ssim_score, parent_crop, downsampled, diff = validate_zoom_consistency(
            zoom2_img, zoom1_img, zoom2_coordinate, zoom1_coordinate
        )
        
        display_images(
            [parent_crop, downsampled, diff],
            ["Parent Crop", "Downsampled Child", f"Difference (SSIM: {ssim_score:.3f})"],
            figsize=(15, 5)
        )
    else:
        print("❌ Failed to generate zoom 2")
else:
    print("⚠️  Skipping Test 3 - no zoom 1 map")

## Summary

**64× Hierarchical Zoom Complete**

All generated images are in `outputs/`:
- `world_64km.png` - Entire world (64km × 64km)
- `zoom1_1km.png` - 64× zoom (1km × 1km)
- `zoom2_16m.png` - 4096× zoom from world (16m × 16m)

**Key Features:**
- ✓ World lore separate from visual style
- ✓ Gemini auto-generates prompts from coordinates
- ✓ 64× zoom levels only
- ✓ First generation is whole world
- ✓ Overlap validation with SSIM metrics
- ✓ Gray borders (bottom+right) for object overflow